In [1]:
from PIL import Image
from PIL import ImageEnhance
from PIL import ImageFilter
import numpy as np
import colorsys
import math
import PIL
from PIL import Image
import imageio
import matplotlib.pyplot as plt
from skimage import io

def bldclrwhl (ny, nx, sym):
    im = Image.new("RGB", (nx,ny))
    radius = min(im.size)/2.0
    centre = im.size[0]/2, im.size[1]/2
    pix = im.load()
    for x in range(im.width):
        for y in range(im.height):
            rx = x - centre[0]
            ry = y - centre[1]
            s=1
            if s <= 1.0:
                h = ((math.atan2(ry, rx) / math.pi) + 1.0) / 2.0
                h = h * sym       #symmetry
                rgb = colorsys.hsv_to_rgb(h, s, 1.0)
                pix[x,y] = tuple([int(round(c*255.0)) for c in rgb])
    imnp = np.array(im)
    return imnp

def nofft(whl, img, nx, ny):
    imnp = np.array(img)
    fimg = np.fft.fft2(imnp)
    whl = np.fft.fftshift(whl)
    proimg = np.zeros((nx,ny,3))
    comb = np.zeros((nx,ny,3), dtype=complex)
    magnitude = np.abs(fimg)
    phase = np.angle(fimg)
    for n in range(3):
        proimg[:,:,n] = whl[:,:,n]*magnitude
        comb[:,:,n] = np.multiply(proimg[:,:,n], np.exp(1j*phase))
        proimg[:, :, n] = np.real(np.fft.ifft2(comb[:,:,n]))
        proimg[:, :, n] = proimg[:, :, n] - np.min(proimg[:, :, n])
        proimg[:, :, n] = proimg[:, :, n] / np.max(proimg[:, :, n])
    return proimg
symmetry = -1
np.save('temp',symmetry)

In [5]:
import base64
import dash
from dash.dependencies import Input, Output, State, ALL
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from flask_caching import Cache
import io as file
from jupyter_dash import JupyterDash
from PIL import Image
import plotly.express as px
import time
from urllib.parse import quote as urlquote

from dash.dependencies import Input, Output
# Load Data
df = px.data.tips()

# App Layout
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

sidebar = html.Div(
    id='slidebar',
    children=[
    html.Div(children='''Symmetry'''),
    dcc.Slider(
        id='my-slider',
        min=1,
        max=12,
        step=1,
        value=6,
        updatemode='mouseup',
        marks={str(n): str(n) for n in range(13)}
    ),
    html.Div(children='''Color Saturation'''),
    dcc.Slider(
        id='color-slider',
        min=0,
        max=20,
        step=0.1,
        value=1,
        updatemode='mouseup'
    ), 
    html.Div(children='''Brightness'''),
    dcc.Slider(
        id='bright-slider',
        min=0,
        max=4,
        step=0.1,
        value=1,
        updatemode='mouseup'
    ), 
    html.Div(children='''Contrast'''),
    dcc.Slider(
        id='contra-slider',
        min=0,
        max=10,
        step=0.1,
        value=1,
        updatemode='mouseup'
    ), 
    html.Div(children='''Blur'''),
    dcc.Slider(
        id='blur-slider',
        min=0,
        max=10,
        step=0.1,
        value=0,
        updatemode='mouseup'
    ),
    html.Div(children='''Overlap'''),
    dcc.Slider(
        id='overlap',
        min=0,
        max=1,
        step=0.1,
        value=1.0,
        updatemode='mouseup'
    )]
)

content = html.Div([
    dcc.Upload(
        id='upload-data',
        children=html.Div(['Drag and Drop or ',
                           html.A('Select Files')
                          ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    html.Div(id='graph'),
    html.Div(id='no-display',
             children=[
                 dcc.Store(id='image-store', data={}),
                 dcc.Store(id='cu_sym', data={}),
                 dcc.Store(id='temp-img', data={}),
                 dcc.Store(id='clrwhl', data={})
             ])
    
])

app.layout = html.Div(children=[html.H1("Colorwheel for Orientation"), 
                                dbc.Row([dbc.Col(sidebar, width=3), 
                                         dbc.Col(content, width=9)]),
                                dcc.Loading(id='download'),
                                # dcc.Download(id='download')
                                ],
                     style={'margin-top': '3rem',
                            'margin-bottom': '3rem',
                            'margin-left': '3rem',
                            'margin-right': '3rem'
                           })


# Reads the image in cache and returns it as a numpy array
def read_img_cache(image_cache):
    # image_cache is a dict, keys=filename, value=bin encoding
    img_bytes = base64.b64decode(image_cache)
    im_bbytes = file.BytesIO(img_bytes)
    im = PIL.Image.open(im_bbytes)
    return np.array(im)


# Returns the figure
def make_figure(image_npy, clrwhl):
    width, height = np.array(image_npy).shape[0:2]
    fig1 = px.imshow(image_npy)
    fig2 = px.imshow(clrwhl)
    return fig1, fig2


# Define callback to upload image(s)
@app.callback(
    Output('image-store', 'data'),
    Output('cu_sym', 'data'),
    Output('graph', 'children'),
    Input('upload-data', 'contents'),
    Input('upload-data', 'filename')
)
def image_upload(upload_image_contents, upload_image_filename):
    if upload_image_contents is None:
        raise PreventUpdate
    start_time = time.time()
    if upload_image_contents is not None:
        image_store_data = {}
        for c, n in zip(upload_image_contents, upload_image_filename):
            content_type, content_string = c.split(',')
            image_store_data[n] = (content_type, content_string)
        image_slider_max = len(upload_image_filename)-1
    contents = [html.Div(id={'type': 'contents', 'index': 0},
                         children=[dcc.Graph(id={'type': 'graph', 'index': 0}),
                                   dcc.Slider(id={'type': 'image-slider', 'index': 0},
                                         min=0,
                                         max=image_slider_max,
                                         value=0,
                                         updatemode='mouseup',
                                         tooltip={"placement": "top", "always_visible": True})],
                         style={'display': 'none'}),
                html.Div(id={'type': 'contents', 'index':1},
                         children=[dcc.Graph(id={'type': 'graph', 'index': 1}),
                                   dbc.Button("SAVE", 
                                              id={'type':'save-data', 'index': 0}, 
                                              className="ms-auto", 
                                              n_clicks=0,
                                              style={'width': '95%'})],
                         style={'display': 'none'}),
                
               ]
    print('Upload time: ', time.time()-start_time)
    return image_store_data, -1, contents


# Define callback to update graph
@app.callback(
    [
        Output({'type': 'graph', 'index': ALL}, 'figure'),
        Output({'type': 'contents', 'index': ALL}, 'style'),
        Output('temp-img', 'data'),
        Output('clrwhl', 'data')
    ],
    [
        Input('my-slider', 'value'),
        Input('color-slider', 'value'),
        Input('bright-slider', 'value'), 
        Input('contra-slider', 'value'), 
        Input('blur-slider', 'value'),
        Input('overlap','value'),
        Input('image-store', 'data'),
        Input({'type': 'image-slider', 'index': ALL}, 'value'),
        State('cu_sym', 'data'),
        State('temp-img', 'data'),
        State('clrwhl', 'data')
    ] 
)
def update_figure(symmetry, enh_val, bright_val, contra_val, blur_val, overlap, image_store_data, slider_value, 
                  cu_sym, rgb2, clrwhl):
    if len(image_store_data)==0:
        raise PreventUpdate
    if cu_sym is not symmetry:
        start_time = time.time()
        print('Reload')
        if slider_value[0] is not None:
            if slider_value[0]<len(image_store_data):
                im_cache = image_store_data[list(image_store_data.keys())[slider_value[0]]][1]
        else:
            im_cache = image_store_data[list(image_store_data.keys())[0]][1]
        im = read_img_cache(im_cache)[:,:,0]
        print('Read data time: ', time.time() - start_time)
        start_time = time.time()
        clrwhl = bldclrwhl(im.shape[0],im.shape[1],symmetry)
        rgb = nofft(clrwhl, im,im.shape[0],im.shape[1])
        iim = np.zeros((im.shape[0],im.shape[1],3))
        iim[:,:,0] = im
        iim[:,:,1] = im
        iim[:,:,2] = im
        test  = read_img_cache(im_cache)[:,:,:]
        rgb2 = test * (1-overlap) + overlap* iim* rgb 
    rgb2 = Image.fromarray(np.uint8(rgb2))
    img2 = rgb2.filter(ImageFilter.GaussianBlur(radius = blur_val)) 
    converter = PIL.ImageEnhance.Color(img2)
    img2 = converter.enhance(enh_val)
    converter = PIL.ImageEnhance.Brightness(img2)
    img2 = converter.enhance(bright_val)
    converter = PIL.ImageEnhance.Contrast(img2)
    img2 = converter.enhance(contra_val)
    print('Processing time: ', time.time() - start_time)
    start_time = time.time()
    graph1, graph2 = make_figure(img2, clrwhl)
    print('Graph generation time: ', time.time() - start_time)
    return [graph1, graph2], [{'width': '55%', 'display': 'inline-block', 'padding': '0 20'}, 
                              {'display': 'inline-block', 'width': '35%'}], rgb2, clrwhl

                                
                                
# Downloads image
@app.callback(
    Output("download", "children"),
    Input({'type': 'save-data', 'index': ALL}, "n_clicks"),
    State({'type': 'graph', 'index': ALL}, 'figure'),
    prevent_initial_call=True,
)
def func(n_clicks, image):
    if n_clicks[0]>0:
        im_cache = image[0]['data'][0]['source']
        filename = 'image.jpeg'
        url = "/download/" + urlquote(filename)
        return html.A(download=filename, href=im_cache, children=["Click here to start download"])
    pass
                                
# Run app and display result inline in the notebook
# app.run_server(mode='inline')
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/


/home/ahexemer/Downloads/conda/envs/ML/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



Upload time:  0.0016777515411376953
Reload
Read data time:  0.026882410049438477
Processing time:  1.5537810325622559
Graph generation time:  0.16910243034362793
Reload
Read data time:  0.033776044845581055
Processing time:  1.5042293071746826
Graph generation time:  0.16924309730529785
Reload
Read data time:  0.016348838806152344
Processing time:  1.557053804397583
Graph generation time:  0.16617035865783691
Reload
Read data time:  0.01603102684020996
Processing time:  1.587522268295288
Graph generation time:  0.1595463752746582
Reload
Read data time:  0.01622915267944336
Processing time:  1.5517544746398926
Graph generation time:  0.1792302131652832
Reload
Read data time:  0.03432750701904297
Processing time:  1.5240325927734375
Graph generation time:  0.1542961597442627
Reload
Read data time:  0.01572728157043457
Reload
Read data time:  0.07452797889709473
Processing time:  3.7202959060668945
Graph generation time:  0.17925143241882324
Processing time:  4.0957465171813965
Graph gene